## Sklearn Decision Tree Regressor 
### We will peform GridSearch to find the optimal parameters to be used in the Decision Tree Regressor model to predict sales. Tuning of the hyperparameters are based on the MSE value obtained

In [1]:
#Importing relevant libraries
from data import train
from sklearn.svm import SVR
import numpy as np
from multiprocessing import cpu_count
from utils import StandardizedGridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split

In [2]:
#Separating predictors from sales column
X=train.drop('sales', axis=1)
y=train.sales

In [9]:
#Retriving saved Decision Tree Models
# Decision Tree Run 1

DT_run1=StandardizedGridSearchCV.load('models/DT1.p')
DT_run1

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(), n_jobs=1,
             param_grid={'ccp_alpha': (0.01, 0.1, 1), 'max_depth': (9, 12),
                         'max_features': (None, 'auto'),
                         'max_leaf_nodes': array([ 199,  629, 1992]),
                         'min_samples_split': (2, 4, 6)},
             refit='neg_mean_squared_error', return_train_score=True,
             scoring=['neg_mean_squared_error', 'r2'], verbose=10)

In [10]:
# Top 10 model results for Decision Tree Run 1

DT_run1.results.head(10)

auto         1992           4                         0.003052   
0.10      9         auto         199            2                         0.002943   
                                 629            4                         0.003528   

                                                                   std_score_time  \
ccp_alpha max_depth max_features max_leaf_nodes min_samples_split                   
0.01      9         auto         199            2                        0.001460   
                                 629            2                        0.002255   
                                                4                        0.005329   
1.00      12        auto         1992           2                        0.003114   
          9         auto         1992           4                        0.003392   
                    NaN          1992           4                        0.004536   
0.01      12        auto         629            4                        0.003541   
          9         auto         1992           4                        0.001492   
0.10      9         auto         199            2                        0.001673   
                                 629            4                        0.003065   

                                                                   split0_test_neg_mean_squared_error  \
ccp_alpha max_depth max_features max_leaf_nodes min_samples_split                                       
0.01      9         auto         199            2                                        -1325.775809   
                                 629            2                                        -1404.310178   
                                                4                                        -1222.140055   
1.00      12        auto         1992           2                                        -1953.219419   
          9         auto         1992           4                                        -1360.113779   
                    NaN          1992           4                                        -1356.719092   
0.01      12        auto         629            4                                        -1278.863311   
          9         auto         1992           4                                        -1235.374623   
0.10      9         auto         199            2                                        -1654.746816   
                                 629            4                                        -1395.044285   

                                                                   split1_test_neg_mean_squared_error  \
ccp_alpha max_depth max_features max_leaf_nodes min_samples_split                                       
0.01      9         auto         199            2                                        -1931.729197   
                                 629            2                                        -1654.479197   
                                                4                                        -1877.503919   
1.00      12        auto         1992           2                                        -1807.163690   
          9         auto         1992           4                                        -1882.268747   
                    NaN          1992           4                                        -1921.699684   
0.01      12        auto         629            4                                        -2114.693353   
          9         auto         1992           4                                        -1838.541419   
0.10      9         auto         199            2                                        -1986.369144   
                                 629            4                                        -1885.298172   

                                                                   split2_test_neg_mean_squared_error  \
ccp_alpha max_depth max_features max_leaf_nodes min_samples_split                                       
0.01    

In [11]:
#Printing top 10 models for Decision Tree Run 1 based on test MSE value

DT_run1.results['mean_test_neg_mean_squared_error'].head(10)

ccp_alpha  max_depth  max_features  max_leaf_nodes  min_samples_split
0.01       9          auto          199             2                   -2875.157479
                                    629             2                   -2877.963026
                                                    4                   -2901.733712
1.00       12         auto          1992            2                   -2910.683916
           9          auto          1992            4                   -2913.025647
                      NaN           1992            4                   -2913.996785
0.01       12         auto          629             4                   -2921.219217
           9          auto          1992            4                   -2950.312994
0.10       9          auto          199             2                   -2956.852905
                                    629             4                   -2968.607560
Name: mean_test_neg_mean_squared_error, dtype: float64

In [12]:
#We aggreagted the train and test MSE scores of run1 to see if max_leaf_nodes had a significant effect on the MSE values

(-DT_run1.results[['mean_train_neg_mean_squared_error', 'mean_test_neg_mean_squared_error']]).groupby('max_leaf_nodes').mean()

,mean_train_neg_mean_squared_error,mean_test_neg_mean_squared_error
max_leaf_nodes,,
199,379.647435,3276.184532
629,371.666809,3275.611145
1992,371.666972,3269.282636


### From above, we can see that max_leaf_nodes have minimal effect on the train and test MSE score

In [14]:
#We aggreagted the train and test MSE scores of run1 to see if ccp_alpha had a significant effect on the MSE values
(-DT_run1.results[['mean_train_neg_mean_squared_error', 'mean_test_neg_mean_squared_error']]).groupby('ccp_alpha').mean()

,mean_train_neg_mean_squared_error,mean_test_neg_mean_squared_error
ccp_alpha,,
0.01,360.260817,3272.226911
0.10,362.370019,3233.586938
1.00,400.350380,3315.264464


### From output above, we decided to tune ccp_alpha to include a wider range, let max_leaf_nodes be unlimited to see if test MSE score will be lowered

In [6]:
# Decision Tree Run 2

DT_run2=StandardizedGridSearchCV.load('models/DT2.p')
DT_run2

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(), n_jobs=1,
             param_grid={'ccp_alpha': array([0.01      , 0.01473684, 0.01947368, 0.02421053, 0.02894737,
       0.03368421, 0.03842105, 0.04315789, 0.04789474, 0.05263158,
       0.05736842, 0.06210526, 0.06684211, 0.07157895, 0.07631579,
       0.08105263, 0.08578947, 0.09052632, 0.09526316, 0.1       ]),
                         'max_depth': [9], 'max_features': (None, 'auto'),
                         'min_samples_split': (4, 6)},
             refit='neg_mean_squared_error', return_train_score=True,
             scoring=['neg_mean_squared_error', 'r2'], verbose=10)

In [7]:
# Top 10 model results for Decision Tree Run 2

DT_run2.results.head(10)

,,,,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_neg_mean_squared_error,split1_test_neg_mean_squared_error,split2_test_neg_mean_squared_error,split3_test_neg_mean_squared_error,split4_test_neg_mean_squared_error,split5_test_neg_mean_squared_error,...,split2_train_r2,split3_train_r2,split4_train_r2,split5_train_r2,split6_train_r2,split7_train_r2,split8_train_r2,split9_train_r2,mean_train_r2,std_train_r2
ccp_alpha,max_depth,max_features,min_samples_split,,,,,,,,,,,,,,,,,,,,,
0.095263,9,auto,4,0.024533,0.001100,0.003295,0.001012,-1632.685951,-1941.067200,-4713.838185,-1303.535837,-2538.609710,-1706.257310,...,0.971578,0.941948,0.943104,0.946356,0.947920,0.943089,0.974900,0.946991,0.950553,0.011612
0.062105,9,NaN,4,0.024734,0.001773,0.002987,0.000782,-1401.122251,-1699.808601,-4939.669540,-1601.733226,-2686.075868,-1578.481374,...,0.971613,0.942008,0.943135,0.946386,0.947953,0.943102,0.974917,0.947050,0.950589,0.011608
0.071579,9,NaN,4,0.021043,0.001308,0.002299,0.000464,-1376.882439,-1714.800499,-4778.802651,-1856.229125,-2840.678889,-1706.631291,...,0.971595,0.941989,0.943135,0.946379,0.947946,0.943096,0.974909,0.947038,0.950578,0.011607
0.043158,9,NaN,4,0.020838,0.001579,0.002502,0.000498,-1294.492161,-1828.496511,-4902.223793,-1516.547555,-2079.927816,-1778.326102,...,0.971622,0.942029,0.943167,0.946412,0.947990,0.943107,0.974964,0.947061,0.950610,0.011612
0.095263,9,auto,6,0.025034,0.002195,0.003287,0.000790,-1325.459911,-1794.902554,-4716.987566,-1499.177117,-3019.475621,-1597.769021,...,0.964700,0.934656,0.929114,0.939612,0.941458,0.938794,0.968603,0.943223,0.943894,0.012030
0.066842,9,auto,6,0.020850,0.001294,0.002297,0.000463,-1329.550396,-1799.180437,-4885.127772,-1484.075666,-2815.542979,-1655.717005,...,0.964700,0.934704,0.929146,0.939635,0.941485,0.938801,0.968612,0.943262,0.943917,0.012021
0.090526,9,auto,6,0.027041,0.001903,0.002890,0.000697,-1327.877809,-1812.733214,-4716.987566,-1486.290424,-3035.564752,-1559.101797,...,0.964700,0.934674,0.929114,0.939621,0.941468,0.938794,0.968603,0.943223,0.943897,0.012028
0.010000,9,auto,6,0.021132,0.002032,0.002901,0.000311,-1326.216360,-1808.369749,-4782.389447,-1498.524886,-2975.698575,-1655.499784,...,0.964743,0.934748,0.929190,0.939655,0.941542,0.938828,0.968681,0.943297,0.943961,0.012026
0.066842,9,NaN,4,0.021230,0.001419,0.002398,0.000659,-1357.078550,-1875.301472,-4787.158458,-1542.517300,-2684.254220,-1706.631291,...,0.971595,0.941996,0.943135,0.946386,0.947946,0.943096,0.974909,0.947038,0.950579,0.011606


In [8]:
#Printing top 10 models for Decision Tree Run 2based on test MSE value

DT_run2.results['mean_test_neg_mean_squared_error'].head(10)

ccp_alpha  max_depth  max_features  min_samples_split
0.095263   9          auto          4                   -2928.737629
0.062105   9          NaN           4                   -2944.705073
0.071579   9          NaN           4                   -2977.647594
0.043158   9          NaN           4                   -2980.986303
0.095263   9          auto          6                   -2991.367124
0.066842   9          auto          6                   -2991.967106
0.090526   9          auto          6                   -2992.914946
0.010000   9          auto          6                   -2996.789787
0.066842   9          NaN           4                   -2999.027430
0.033684   9          auto          6                   -3000.834978
Name: mean_test_neg_mean_squared_error, dtype: float64

### Comparing the 2 models above, DT_run1 is the better model as it gives a lower test MSE score

In [15]:
## Printing best parameters from DT_run1

DT_run1.best_estimator_

DecisionTreeRegressor(ccp_alpha=0.01, max_depth=9, max_features='auto',
                      max_leaf_nodes=199)